In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install package dependencies


In [ ]:
import numpy as np
import os
random_seed = 42  # for reproducibility
import tensorflow as tf
from tensorflow.keras.optimizers import *
# import matplotlib.pyplot as plt
from math import sqrt
from GCN.data_processing import Data_Loader
from GCN.graph import Graph
from GCN.sgcn_lstm import Sgcn_Lstm
import pandas as pd

In [ ]:
def calculate_accuracy(y_true, y_pred):
    return np.sum(y_true == y_pred) / len(y_true)

def calculate_F1_score(y_true, y_pred):
    TP = 0
    FP = 0
    FN = 0
    for t, this_label in enumerate(y_pred):
        if this_label == 1:
            if this_label == y_true[t]:
                TP += 1
            else:
                FP += 1
        elif this_label != y_true[t]:
            FN += 1

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    F1 = 2. * (precision * recall) / (precision + recall)
    return F1, precision, recall

In [ ]:
# Go through k-fold validation and load/apply all networks then store in pandas dataframe
data_dir = "/content/"
networks_dir = "/content/"

# Set up columns for the dataframe
accuracies = []
precisions = []
recalls = []
F1s = []
folds = []
exercises = []

# Storage for predictions and GT labels, list of lists of shape [n_folds, n_exercises, 3, n_test_samples]
# shape 3 comes from [gt labels, predicted probabilities, predicted classes
results = []
for this_fold in os.listdir(data_dir):
    fold_folder = os.path.join(data_dir, this_fold)
    if not os.path.isdir(this_fold):
        continue
    results.append([])
    for this_network in os.listdir(fold_folder):
        if os.path.splitext(this_network)[1] != '.hdf5':
            continue

        network_path = os.path.join(networks_dir, this_fold, this_network)

        test_data_loader = Data_Loader(data_dir, False)  # load the data
        graph = Graph(len(test_data_loader.body_part))  # create the graph data structure
        test_x, test_y = test_data_loader.scaled_x, test_data_loader.scaled_y  # get the scaled testing data

        # create the model and load the weights
        algorithm = Sgcn_Lstm(test_x, test_y, graph.AD, graph.AD2, graph.bias_mat_1, graph.bias_mat_2)
        algorithm.create_model()
        algorithm.model.load_weights(network_path)

        # apply the model
        y_probs = algorithm.prediction(test_x)
        y_pred = np.argmax(y_probs, axis=1)
        test_y = np.squeeze(test_y)
        test_y = np.argmax(test_y, axis=1)
        results[-1].append([test_y, y_probs, y_pred])

        # make some measurements
        acc = calculate_accuracy(test_y, y_pred)
        F1, precision, recall = calculate_F1_score(test_y, y_pred)

        folds.append(int(this_fold.split("_")[1]))
        exercises.append(os.path.splitext(this_network)[0-].split("_")[1])
        accuracies.append(acc)
        precisions.append(precision)
        recalls.append(recall)
        F1s.append(F1)

# Combine into a pandas dataframe
df = pd.DataFrame({"fold": folds, "exercise": exercises, "precision": precisions, "recall": recalls, "accuracy": accuracies, "F1": F1s})